## Code for annotating the TiDy QA

In [2]:
import pandas as pd
import gc
from datasets import load_dataset, Dataset

/home/robin/.cache/pypoetry/virtualenvs/qtype-eval-pAepV5Z2-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [38]:
pd.set_option('display.max_colwidth', 1000)
#pd.set_option('display.max_rows', 100)

In [4]:


ds = load_dataset("google-research-datasets/tydiqa", "primary_task")


In [71]:
ds.set_format("pandas")
ds['validation'][:5]

#df = ds['validation'][:5]
#print(df['annotations'].to_string())
print(len(ds['train'])+len(ds['validation']))

185586


In [39]:
ds.set_format("pandas")
df = ds['validation'][:5]



print(df['annotations'].to_string())

0          {'passage_answer_candidate_index': [-1, -1, -1], 'minimal_answers_start_byte': [-1, -1, -1], 'minimal_answers_end_byte': [-1, -1, -1], 'yes_no_answer': ['NONE', 'NONE', 'NONE']}
1          {'passage_answer_candidate_index': [-1, -1, -1], 'minimal_answers_start_byte': [-1, -1, -1], 'minimal_answers_end_byte': [-1, -1, -1], 'yes_no_answer': ['NONE', 'NONE', 'NONE']}
2    {'passage_answer_candidate_index': [27, -1, -1], 'minimal_answers_start_byte': [39087, -1, -1], 'minimal_answers_end_byte': [39089, -1, -1], 'yes_no_answer': ['NONE', 'NONE', 'NONE']}
3      {'passage_answer_candidate_index': [-1, 16, -1], 'minimal_answers_start_byte': [-1, 8154, -1], 'minimal_answers_end_byte': [-1, 8158, -1], 'yes_no_answer': ['NONE', 'NONE', 'NONE']}
4          {'passage_answer_candidate_index': [-1, -1, -1], 'minimal_answers_start_byte': [-1, -1, -1], 'minimal_answers_end_byte': [-1, -1, -1], 'yes_no_answer': ['NONE', 'NONE', 'NONE']}


In [15]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', 100)  # Set maximum column width
pd.set_option('display.max_rows', 10)       # Limit number of rows displayed
pd.set_option('display.width', 100)        # Set display width

In [29]:

ds.set_format("pandas")

df = ds['validation'][:].copy()




polar_filter = df['annotations'].apply(lambda x: all(ans not in ['NONE','NO'] for ans in x['yes_no_answer']))


polar = df[polar_filter]
wh = df[~polar_filter]

polar_questions = polar[['question_text', 'language', 'annotations']]
wh_questions = wh[['question_text', 'language', 'annotations']]

polar_ds = Dataset.from_pandas(polar_questions)
print(polar_ds)


wh_ds = Dataset.from_pandas(wh_questions)
print(wh_ds)

Dataset({
    features: ['question_text', 'language', 'annotations', '__index_level_0__'],
    num_rows: 551
})
Dataset({
    features: ['question_text', 'language', 'annotations', '__index_level_0__'],
    num_rows: 18119
})


In [34]:
polar_ds.set_format('pandas')
wh_ds.set_format('pandas')
polar_ds[:5]
wh_ds[:5]

,question_text,language,annotations,__index_level_0__
0,หม่อมราชวงศ์สุขุมพันธุ์ บริพัตร เรียนจบจากที่ไหน ?,thai,"{'minimal_answers_end_byte': [-1, -1, -1], 'minimal_answers_start_byte': [-1, -1, -1], 'passage_...",0
1,Ukubwa wa Rijili Kantori ni kiasi gani?,swahili,"{'minimal_answers_end_byte': [-1, -1, -1], 'minimal_answers_start_byte': [-1, -1, -1], 'passage_...",1
2,అలి గౌహర్ ఎన్ని సంవత్సరాలు జీవించాడు?,telugu,"{'minimal_answers_end_byte': [39089, -1, -1], 'minimal_answers_start_byte': [39087, -1, -1], 'pa...",2
3,Milloin Kokemäki on perustettu?,finnish,"{'minimal_answers_end_byte': [-1, 8158, -1], 'minimal_answers_start_byte': [-1, 8154, -1], 'pass...",3
4,১৯৮১ সালে বাংলাদেশের সেনাবাহিনীর সেনাপ্রধান কে ছিলেন ?,bengali,"{'minimal_answers_end_byte': [-1, -1, -1], 'minimal_answers_start_byte': [-1, -1, -1], 'passage_...",4


In [85]:
from datasets import load_from_disk

validation_polar = load_from_disk("/home/robin/Research/qtype-eval/tydiqa-validation-polar")
validation_wh = load_from_disk("/home/robin/Research/qtype-eval/tydiqa-validation-wh")
train_polar = load_from_disk("/home/robin/Research/qtype-eval/tydiq-train-polar")
train_wh = load_from_disk("/home/robin/Research/qtype-eval/tydiqa-train-wh")

validation_polar.set_format('pandas')
validation_wh.set_format('pandas')
train_polar.set_format('pandas')
train_wh.set_format('pandas')
#validation_polar[:5]
#validation_wh[100:110]
#train_polar[:5]
#train_wh[45:50]



In [86]:
print(len(train_wh))
print(len(train_polar))
print(len(validation_polar))
print(len(validation_wh))

97390
5222
735
10595


In [87]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

def get_language_counts(df, name):
    counts = df[:]['language'].value_counts()
 
    return counts

# Print counts for each split
polar_valid_counts = get_language_counts(validation_polar, "Polar Questions - Validation Split")
wh_valid_counts = get_language_counts(validation_wh, "WH Questions - Validation Split")
polar_train_counts = get_language_counts(train_polar, "Polar Questions - Training Split")
wh_train_counts = get_language_counts(train_wh, "WH Questions - Training Split")

# Create summary DataFrame
summary = pd.DataFrame({
    'Polar-Valid': polar_valid_counts,
    'WH-Valid': wh_valid_counts,
    'Polar-Train': polar_train_counts,
    'WH-Train': wh_train_counts
})

print("\n=== Complete Summary ===")
print(summary)




=== Complete Summary ===
            Polar-Valid  WH-Valid  Polar-Train  WH-Train
language                                                
arabic               98      1282         1179     21913
english              77       954          444      8767
finnish              94      1988          950     14335
indonesian           64      1741          328     14624
japanese            142      1567         1031     15257
korean               65      1633          296     10685
russian             195      1430          994     11809


## Memory hygiene

In [88]:
pd.DataFrame.cache = {}
gc.collect()

587